In [16]:
from openmm.app import *
from openmm import *
import numpy as np
import mdtraj as md
from itertools import combinations
import sys


pdb_file = "./eq-par.pdb"
psf_file = "./complex.psf"

top_inp = ['./top_hyres_GPU.inp', 'top_atp.inp']
param_inp = ['param_hyres_GPU.inp', 'param_atp.inp']

temp = 380
dt = 0.002*unit.picoseconds                                     # time step
total_step = 4000000000  #     8 us                              # total step
temperature = temp*unit.kelvin                                    # temperature
log_freq = 10000                                                 # frequency of log file
dcd_freq = 100000 # 50000                                                 # frequency of fine dcd file
c_ion = 0.15                                                    # concentration of ions

lx = 25.0*unit.nanometer                                        # pbc box length
a = Vec3(lx, 0.0, 0.0)
b = Vec3(0.0, lx, 0.0)
c = Vec3(0.0, 0.0, lx)

er = 20.0                                                       # effective dielectric constant
eps_hb = 2.0*unit.kilocalorie_per_mole                          # hydrogen bond strength
sigma_hb = 0.29*unit.nanometer                                  # sigma of hydrogen bond
r_cut = 1.8*unit.nanometer                                      # cutoff distance of nonbondedforce
pressure = 1*unit.atmosphere                                    # pressure in NPT
friction = 0.1/unit.picosecond                                    # friction coefficient in Langevin


pdb_eq = PDBFile(pdb_file)
pdb_init = PDBFile(pdb_file)
psf = CharmmPsfFile(psf_file)
psf.setBox(lx, lx, lx)
top = psf.topology
params = CharmmParameterSet(*top_inp, *param_inp)
system = psf.createSystem(params, nonbondedMethod=CutoffPeriodic, constraints=HBonds)
system.setDefaultPeriodicBoxVectors(a, b, c)

# load hyres force field

In [17]:
for force in system.getForces():
    if force.getName() == "NonbondedForce":
        nbforce = force
print('get the NonBondedForce:', nbforce.getName())

# add custom nonbondedforce: CNBForce
formula = '(step(Ron - r) + '+ \
'(step(r - Ron) * step(Roff - r) - '+ \
'step(r - Ron) * step(Ron - r)) * '+ \
'(((Roff2 - r2)^2 * (Roff2 + 2.0 * r2 - 3.0 * Ron2)) / '+ \
'(Roff2 - Ron2)^3)) * '+ \
'(4.0 * epsilon * six * (six - 1.0) + (138.935456 / eps * charge1 * charge2) / r * exp(-kf * r));'+ \
'six = (sigma / r)^6; '+ \
'sigma = 0.5 * (sigma1 + sigma2); '+ \
'epsilon = sqrt(epsilon1 * epsilon2);'+ \
'Ron2 = Ron * Ron; Roff2 = Roff * Roff; r2 = r * r; '
CNBForce = CustomNonbondedForce(formula)
CNBForce.setNonbondedMethod(nbforce.getNonbondedMethod())
CNBForce.addGlobalParameter('eps', er)
CNBForce.addGlobalParameter('Ron', r_cut - 0.2*unit.nanometer)
CNBForce.addGlobalParameter('Roff', r_cut)
CNBForce.addGlobalParameter('kf', np.sqrt(c_ion/9.480)*AngstromsPerNm)
CNBForce.setCutoffDistance(r_cut)

# perparticle variables: sigma, epsilon, charge,
CNBForce.addPerParticleParameter('charge')
CNBForce.addPerParticleParameter('sigma')
CNBForce.addPerParticleParameter('epsilon')
for idx in range(nbforce.getNumParticles()):
    particle = nbforce.getParticleParameters(idx)
    perP = [particle[0], particle[1], particle[2]]
    CNBForce.addParticle(perP)
# add exclusion
for idx in range(nbforce.getNumExceptions()):
    ex = nbforce.getExceptionParameters(idx)
    CNBForce.addExclusion(ex[0], ex[1])
system.addForce(CNBForce)


# add nonbondedforce of 1-4 interaction through custombondforece
formula = '(step(Ron - r) + '+ \
'(step(r - Ron) * step(Roff - r) - '+ \
'step(r - Ron) * step(Ron - r)) * '+ \
'(((Roff2 - r2)^2 * (Roff2 + 2.0 * r2 - 3.0 * Ron2)) / '+ \
'(Roff2 - Ron2)^3)) * '+ \
'(4.0 * epsilon * six * (six - 1.0) + (138.935456 / eps * charge) / r * exp(-kf * r));'+ \
'six = (sigma / r)^6; '+ \
'Ron2 = Ron * Ron; Roff2 = Roff * Roff; r2 = r * r; '
Force14 = CustomBondForce(formula)
Force14.addGlobalParameter('eps', er)
Force14.addGlobalParameter('Ron', r_cut - 0.2*unit.nanometer)
Force14.addGlobalParameter('Roff', r_cut)
Force14.addGlobalParameter('kf', np.sqrt(c_ion/9.480)*10.00)

Force14.addPerBondParameter('charge')
Force14.addPerBondParameter('sigma')
Force14.addPerBondParameter('epsilon')
for idx in range(nbforce.getNumExceptions()):
    ex = nbforce.getExceptionParameters(idx)
    Force14.addBond(ex[0], ex[1], [ex[2], ex[3], ex[4]])
system.addForce(Force14)

# Add the Custom hydrogen bond force
formula  = 'epsilon*(5.0*(sigma/r)^12-6.0*(sigma/r)^10)*swrad*cosd^4*swang; '+ \
'swrad = step(rcuton-r)+step(r-rcuton)*(step(rcutoff-r)-step(rcuton-r))*'+ \
'roff2*roff2*(roff2-3.0*ron2)/roffon2^3; '+ \
'roff2 = rcutoff*rcutoff-r*r; '+ \
'ron2 = rcuton*rcuton-r*r; '+ \
'roffon2 = rcutoff*rcutoff-rcuton*rcuton; '+ \
'rcutoff = CTOFHB; rcuton = CTONHB; r = distance(a1, d2); '+ \
'swang = step(cosdcuton-cosd)+step(cosd-cosdcuton)*(step(cosdcutoff-cosd)-step(cosdcuton-cosd))*'+ \
'cosdoff2*cosdoff2*(cosdoff2-3.0*cosdon2)/cosdoffon2^3; '+ \
'cosdoff2 = cosdcutoff*cosdcutoff-cosd*cosd; '+ \
'cosdon2 = cosdcuton*cosdcuton-cosd*cosd; '+ \
'cosdoffon2 = cosdcutoff*cosdcutoff-cosdcuton*cosdcuton; '+ \
'cosdcutoff = -cos(CTOFHA); cosdcuton = -cos(CTONHA); cosd = cos(angle(a1,d1,d2));'

Hforce = CustomHbondForce(formula)
Hforce.setNonbondedMethod(nbforce.getNonbondedMethod())
Hforce.addGlobalParameter('CTOFHB', 0.5*unit.nanometer)
Hforce.addGlobalParameter('CTONHB', 0.4*unit.nanometer)
Hforce.addGlobalParameter('CTOFHA', 91*unit.degree)
Hforce.addGlobalParameter('CTONHA', 90*unit.degree)
Hforce.addGlobalParameter('sigma', sigma_hb)
Hforce.addGlobalParameter('epsilon', eps_hb)
Hforce.setCutoffDistance(0.6*unit.nanometer)

Ns, Hs, Os, Cs = [], [], [], []
for atom in psf.topology.atoms():
    if atom.name == "N" and atom.residue.name != 'PRO':
        Ns.append(int(atom.index))
    if atom.name == "H":
        Hs.append(int(atom.index))
    if atom.name == "O":
        Os.append(int(atom.index))
    if atom.name == "C":
        Cs.append(int(atom.index))

for idx in range(len(Hs)):
    Hforce.addDonor(Hs[idx], Ns[idx], -1)
    Hforce.addAcceptor(Os[idx], -1, -1)
system.addForce(Hforce)

# delete the NonbondedForce
system.removeForce(nbforce.getForceGroup())

get the NonBondedForce: NonbondedForce


# my restraint functions

In [18]:
from openmm.app import *
from openmm import *
import numpy as np
import mdtraj as md
from itertools import combinations

# helper function to get atom indices for a give pdb
def get_atom_indices_coordinates(pdb, selection):
    """
    pdb: pdb file
    selection: mdtraj selection syntax (ref: https://mdtraj.org/1.9.4/atom_selection.html)
        of note, if your 'selection' has chainid, you need to use the chainid from PDB file.
            chainid in mdtraj is forced to be numbered as 0, 1, 2, ...
            here, instead of using those sequential integers for chainid, we override it with the 
            original chain id from PDB file.
    """
    pdb_md = md.load_pdb(pdb)

    if 'chainid' in selection:
        chainid_dict = {chain.chain_id: chain.index for chain in pdb_md.topology.chains}

        new_str = []
        for substring in selection.split('and'):
            if 'chainid' in substring:
                chainid_list = substring.strip().split('chainid')[-1].strip().split()
                new_chainid_str = ' '.join([str(chainid_dict[i]) for i in chainid_list])
                final_str = 'chainid ' + new_chainid_str
                new_str.append(final_str)
            else:
                new_str.append(substring.strip())
        new_selection = ' and '.join(new_str)
    else:
        new_selection = selection
    
    indices = pdb_md.topology.select(new_selection)
    coordinates = pdb_md.xyz[0, indices, :]
    return indices, coordinates 

def get_COM(pdb, selection):
    pdb_md = md.load_pdb(pdb)

    if 'chainid' in selection:
        chainid_dict = {chain.chain_id: chain.index for chain in pdb_md.topology.chains}
        # print(chainid_dict)

        new_str = []
        for substring in selection.split('and'):
            if 'chainid' in substring:
                chainid_list = substring.strip().split('chainid')[-1].strip().split()
                new_chainid_str = ' '.join([str(chainid_dict[i]) for i in chainid_list])
                final_str = 'chainid ' + new_chainid_str
                new_str.append(final_str)
            else:
                new_str.append(substring.strip())
        new_selection = ' and '.join(new_str)
    else:
        new_selection = selection
    # print(new_selection)
    com = md.compute_center_of_mass(pdb_md, new_selection)[0]
    return com

# Positional restraints
# Positional restraints
def positional_restraint(system, indx_pos_Kcons_list):
    """
    indx_pos_list: list of tuples. Each tuple has two elements, the first one being the atom index,
        the second one being the 3D coordinate (x, y, z) as the reference position

        example: [(1, (10.0, 12.0, 13.0), 400), (10, (20.0, 23.5, 5.6), 400), ...]
            restraint the atom with index 1 to be at (10.0, 12.0, 13.0) with Kcons=400 kj/mol/nm**2; and
            restraint the atom with index 10 to be at (20.0, 23.5, 5.6) with Kcons=400 kj/mol/nm**2
    """

    # omm positional restraints
    pos_restraint = CustomExternalForce('kp*periodicdistance(x, y, z, x0, y0, z0)^2')
    pos_restraint.addPerParticleParameter('kp')
    pos_restraint.addPerParticleParameter('x0')
    pos_restraint.addPerParticleParameter('y0')
    pos_restraint.addPerParticleParameter('z0')
    for idx, pos, Kcons in indx_pos_Kcons_list:
        # add unit to Kcons
        Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
        pos_restraint.addParticle(idx, [Kcons_wU, *pos])
    system.addForce(pos_restraint)

    return system

def bb_positional_restraint(system, pdb_ref, domain=None, Kcons=400):
    """
    add backbone restraints, given a reference PDB
    arguments:
    system: omm system object
    pdb_ref: pdb file path
    domain_ranges: if only wants to restraint backbone of a specific domain
        <-- a new feature to add in future...
    Kcons: (float) K constant. kj/mol/nm**2

    Return: system
    """
    # Load pdb
    pdb_init = PDBFile(pdb_ref)

    # add unit to Kcons
    Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
    
    # omm positional restraints
    pos_restraint = CustomExternalForce('kp*periodicdistance(x, y, z, x0, y0, z0)^2')
    pos_restraint.addGlobalParameter('kp', Kcons_wU)
    pos_restraint.addPerParticleParameter('x0')
    pos_restraint.addPerParticleParameter('y0')
    pos_restraint.addPerParticleParameter('z0')
    for res in pdb_init.topology.residues():
        for atom in res.atoms():
            if atom.name in [ 'CA', 'N', 'C', 'O']:
                pos_restraint.addParticle(atom.index, pdb_init.positions[atom.index])
    system.addForce(pos_restraint)
    return system

# Center of mass positional restraints
def COM_positional_restraint(system, group_indices_pos_kcons):
    """
    system: omm system
    group_indices_pos: list of tuples
        example: [((1,2,3,4), (x0, y0, z0), 400), ...]
            1,2,3,4 are the atom indices of the group; 
            x0, y0, z0 are the x y z coordinates of reference COM position
            400 is the K constant (unit: kj/mol/nm**2) 
    return: omm system    
    """
    # omm positional restraints
    com_pos_restraint = CustomCentroidBondForce(1, 'kp*periodicdistance(x, y, z, x0, y0, z0)^2')
    com_pos_restraint.addPerBondParameter('kp')
    com_pos_restraint.addPerBondParameter('x0')
    com_pos_restraint.addPerBondParameter('y0')
    com_pos_restraint.addPerBondParameter('z0')

    for idx, (group_idx, ref_pos, Kcons) in enumerate(group_indices_pos_kcons):
        Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
        com_pos_restraint.addGroup(group_idx)
        com_pos_restraint.addBond([idx, ], [Kcons_wU, *ref_pos])
    system.addForce(com_pos_restraint)
    return system

# Center of mass distance restraints
def COM_relative_restraint(system, groups_dist_Kcons_list):
    """ Apply distance restraint between two atom groups
    system: omm system
    groups_dist_Kcons_list: (list of tuples)
        example: [((1,2,3), (4,5,6), 10.0, 400), ...]
            (1,2,3): atom indices of atom group1
            (4,5,6): atom indices of atom group2
            1.0: reference distance, unit of nanometer
            400: K constant, kj/mol/nm**2

            restrain distance between atom group 1 and atom group 2 to be 10 Angstrome using Kcons=400 kj/mol/nm**2

    return omm system     
    """
    COM_force = CustomCentroidBondForce(2, "0.5*kp*( (distance(g1, g2)-d0 )^2)")
    COM_force.addPerBondParameter('kp')  # Force constant
    COM_force.addPerBondParameter('d0')  # restrain distance

    i = 0
    for group1,group2,dist,Kcons in groups_dist_Kcons_list:
        Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
        target_dist = dist * unit.nanometers
        COM_force.addGroup(group1)  # Group 1
        COM_force.addGroup(group2)  # Group 2
        COM_force.addBond([i,  i+1], [Kcons_wU, target_dist])
        i += 2
    system.addForce(COM_force)
    return system

# Identify regions with secondary structure
def identify_folded_CA_idx(pdb, domain):
    """
    pdb: mdtraj object
    domain: 
        example: ('A', (1, 50))
    return: CA atom indices for the folded region
    """
    # get chainid dict
    chainid_dict = {chain.chain_id: chain.index for chain in pdb.topology.chains}

    # get starting residue index and ending residue index from domain definition
    chainid, (starting_resid, ending_resid) = domain

    # make sure the given chain id is included in the PDB file
    assert chainid in chainid_dict.keys(), f"chain ID '{chainid}' given in the domain definition does not exist!"

    # select domain
    selection = "chainid %s and residue %s to %s" % (chainid_dict[chainid], starting_resid, ending_resid)
    selected_domain = pdb.atom_slice(pdb.topology.select(selection))
    dssp = md.compute_dssp(selected_domain, simplified=True)
    folded = np.where(dssp!='C', 1, 0)[0]
    resid = [selected_domain.topology.residue(idx).resSeq for idx, i in enumerate(folded) if i==1 ]
    folded_CA_idx = [pdb.topology.select("chainid %s and residue %s and name CA " % (chainid_dict[chainid], str(i) ) )[0] for i in resid]

    return folded_CA_idx

# Domain restraints
def domain_3D_restraint(system, pdb_ref, domain_ranges, Kcons=400, cutoff=1.2):
    """
    system: omm system
    domain_ranges: a list of tuples, and each tuple defines one domain range:
        ('chain_id'), (starting_resid, ending_resid))
        example: [('A', (1,50)), ('B', (75, 200)), ...] 
            # two domains: resid 1-50 of chain A and resid 75-200 of chain B
    Kcons_internal: (float) K constant for harmonic restraint. default unit: kj/mol/nm**2
    cutoff: cutoff distance, unit: nanometer
    return omm system
    """
    internal_force = HarmonicBondForce()
    Kcons_internal = Kcons * unit.kilojoule_per_mole/unit.nanometers**2

    # load pdb to mdtraj
    pdb_md = md.load_pdb(pdb_ref)

    # find all pairs
    pairs = []
    for domain in domain_ranges:
        # get C-alpha atom indices of folder region
        folded_CA_idx = identify_folded_CA_idx(pdb=pdb_md, domain=domain)
        pairs = list(combinations(folded_CA_idx, 2))
    
        pairs_num = 0
        for index in pairs:
            r1=np.squeeze(pdb_md.xyz[:,int(index[0]),:])
            r2=np.squeeze(pdb_md.xyz[:,int(index[1]),:])
            # dist0=np.sqrt((r1[0]-r2[0])**2+(r1[1]-r2[1])**2+(r1[2]-r2[2])**2)
            dist0 = np.linalg.norm(r1-r2)
            if dist0 < 1.2:
                pairs_num += 1
                internal_force.addBond(int(index[0]),int(index[1]), dist0*unit.nanometers, Kcons_internal)
        print(f"Number of internal pairs of domain {str(domain)}: {pairs_num}")
    system.addForce(internal_force)
    
    return system


# Testing positional restraints

In [5]:
def get_atom_indices_coordinates(pdb, selection):
    """
    pdb: pdb file
    selection: mdtraj selection syntax (ref: https://mdtraj.org/1.9.4/atom_selection.html)
        of note, if your 'selection' has chainid, you need to use the chainid from PDB file.
            chainid in mdtraj is forced to be numbered as 0, 1, 2, ...
            here, instead of using those sequential integers for chainid, we override it with the 
            original chain id from PDB file.
    """
    pdb_md = md.load_pdb(pdb)

    if 'chainid' in selection:
        chainid_dict = {chain.chain_id: chain.index for chain in pdb_md.topology.chains}

        new_str = []
        for substring in selection.split('and'):
            if 'chainid' in substring:
                chainid_list = substring.strip().split('chainid')[-1].strip().split()
                new_chainid_str = ' '.join([str(chainid_dict[i]) for i in chainid_list])
                final_str = 'chainid ' + new_chainid_str
                new_str.append(final_str)
            else:
                new_str.append(substring.strip())
        new_selection = ' and '.join(new_str)
    else:
        new_selection = selection
    
    indices = pdb_md.topology.select(new_selection)
    coordinates = pdb_md.xyz[0, indices, :]
    return indices, coordinates 

In [6]:
import copy
system1 = copy.deepcopy(system)

In [7]:
atom_idx, atom_coord = get_atom_indices_coordinates(pdb_file, "chainid A and resid 20 30 40 and name CA")
print(atom_idx, atom_coord)

[130 192 250] [[11.9769  9.3429 12.5621]
 [12.5887  7.6912 12.8775]
 [11.9861  8.1262 11.5796]]


In [8]:
pos_res = []
kcons = 400
for idx,coord in zip(atom_idx, atom_coord):
    # print(idx,coord,kcons)
    pos_res.append((idx,coord,kcons))
print(pos_res)

[(130, array([11.9769,  9.3429, 12.5621], dtype=float32), 400), (192, array([12.5887,  7.6912, 12.8775], dtype=float32), 400), (250, array([11.9861,  8.1262, 11.5796], dtype=float32), 400)]


In [8]:
system1 = positional_restraint(system1, indx_pos_Kcons_list=pos_res)

In [9]:
# save system xml out
with open(f"system1_pos_res_test.xml", "w") as f:
    f.write(XmlSerializer.serialize(system1))

# Testing COM positional restraint

In [52]:
# Center of mass positional restraints
def COM_positional_restraint(system, group_indices_pos_kcons):
    """
    system: omm system
    group_indices_pos: list of tuples
        example: [((1,2,3,4), (x0, y0, z0), 400), ...]
            1,2,3,4 are the atom indices of the group; 
            x0, y0, z0 are the x y z coordinates of reference COM position
            400 is the K constant (unit: kj/mol/nm**2) 
    return: omm system    
    """
    # omm positional restraints
    com_pos_restraint = CustomCentroidBondForce(1, 'kp*periodicdistance(x, y, z, x0, y0, z0)^2')
    com_pos_restraint.addPerBondParameter('kp')
    com_pos_restraint.addPerBondParameter('x0')
    com_pos_restraint.addPerBondParameter('y0')
    com_pos_restraint.addPerBondParameter('z0')

    for idx, (group_idx, ref_pos, Kcons) in enumerate(group_indices_pos_kcons):
        Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
        com_pos_restraint.addGroup(group_idx)
        com_pos_restraint.addBond([idx, ], [Kcons_wU, *ref_pos])
    system.addForce(com_pos_restraint)
    return system

In [44]:
idx1, coord1 = get_atom_indices_coordinates(pdb_file, selection="chainid A and residue 1 to 100")
idx2, coord2 = get_atom_indices_coordinates(pdb_file, selection="chainid B and residue 200 to 250")

com1 = get_COM(pdb_file, selection="chainid A and residue 1 to 100")
com2 = get_COM(pdb_file, selection="chainid B and residue 200 to 250")
print(com1)
print(com2)

[12.50948891  9.0789616  11.77860424]
[12.04790628 12.48202871 17.37442281]


In [46]:
test_com_res_list = [(idx1, com1, 400), (idx2, com2, 400)]

In [47]:
system2 = copy.deepcopy(system)
system2 = COM_positional_restraint(system2, group_indices_pos_kcons=test_com_res_list)
# save system xml out
with open(f"system2_com_pos_res_test.xml", "w") as f:
    f.write(XmlSerializer.serialize(system2))

In [28]:
pdb_test = md.load_pdb(pdb_file)

In [39]:
pdb_test.topology.select("chainid 1 and residue 1 to 1000")

array([3989, 3990, 3991, ..., 7975, 7976, 7977])

# Testing COM distance restraints

In [39]:
def COM_relative_restraint(system, groups_dist_Kcons_list):
    """ Apply distance restraint between two atom groups
    system: omm system
    groups_dist_Kcons_list: (list of tuples)
        example: [((1,2,3), (4,5,6), 10.0, 400), ...]
            (1,2,3): atom indices of atom group1
            (4,5,6): atom indices of atom group2
            1.0: reference distance, unit of nanometer
            400: K constant, kj/mol/nm**2

            restrain distance between atom group 1 and atom group 2 to be 10 Angstrome using Kcons=400 kj/mol/nm**2

    return omm system     
    """
    COM_force = CustomCentroidBondForce(2, "0.5*kp*( (distance(g1, g2)-d0 )^2)")
    COM_force.addPerBondParameter('kp')  # Force constant
    COM_force.addPerBondParameter('d0')  # restrain distance

    i = 0
    for group1,group2,dist,Kcons in groups_dist_Kcons_list:
        Kcons_wU = Kcons * unit.kilojoule_per_mole/unit.nanometers**2
        target_dist = dist * unit.nanometers
        COM_force.addGroup(group1)  # Group 1
        COM_force.addGroup(group2)  # Group 2
        COM_force.addBond([i,  i+1], [Kcons_wU, target_dist])
        i += 2
    system.addForce(COM_force)
    return system

In [33]:
idx1, coord1 = get_atom_indices_coordinates(pdb_file, selection="chainid A and residue 1 to 10 and name CA")
idx2, coord2 = get_atom_indices_coordinates(pdb_file, selection="chainid B and residue 200 to 210 and name CA")

com1 = get_COM(pdb_file, selection="chainid A and residue 1 to 10 and name CA")
com2 = get_COM(pdb_file, selection="chainid B and residue 200 to 210 and name CA")
print(com1)
print(com2)

dist12 = np.linalg.norm(com1-com2)
print(dist12)

idx3, coord3 = get_atom_indices_coordinates(pdb_file, selection="chainid A and residue 200 to 210 and name CA")
idx4, coord4 = get_atom_indices_coordinates(pdb_file, selection="chainid B and residue 1 to 10 and name CA")
com3 = get_COM(pdb_file, selection="chainid A and residue 200 to 210 and name CA")
com4 = get_COM(pdb_file, selection="chainid B and residue 1 to 10 and name CA")
print(com3)
print(com4)

dist34 = np.linalg.norm(com3-com4)
print(dist34)

[11.79698009 10.99487991 10.85535021]
[12.06156358 12.04217269 17.30738189]
6.541829973829677
[13.38658177  9.53274536  9.84988187]
[12.25239    11.7155201  13.42980013]
4.343582769005289


In [34]:
groups_dist_Kcons_list_test = [
    (idx1, idx2, dist12, 400),
    (idx3, idx4, dist34, 400),
]

print(groups_dist_Kcons_list_test)

[(array([ 2,  8, 14, 20, 26, 32, 38, 44, 52, 60]), array([5215, 5221, 5226, 5232, 5238, 5244, 5250, 5256, 5262, 5268, 5274]), 6.541829973829677, 400), (array([1226, 1232, 1237, 1243, 1249, 1255, 1261, 1267, 1273, 1279, 1285]), array([3991, 3997, 4003, 4009, 4015, 4021, 4027, 4033, 4041, 4049]), 4.343582769005289, 400)]


In [40]:
system3 = copy.deepcopy(system)
system3 = COM_relative_restraint(system3, groups_dist_Kcons_list=groups_dist_Kcons_list_test)
# save system xml out
with open(f"system3_com_dist_res_test.xml", "w") as f:
    f.write(XmlSerializer.serialize(system3))

# Testing domain restraint

In [48]:
def domain_3D_restraint(system, pdb_ref, domain_ranges, Kcons=400, cutoff=1.2):
    """
    system: omm system
    domain_ranges: a list of tuples, and each tuple defines one domain range:
        ('chain_id'), (starting_resid, ending_resid))
        example: [('A', (1,50)), ('B', (75, 200)), ...] 
            # two domains: resid 1-50 of chain A and resid 75-200 of chain B
    Kcons_internal: (float) K constant for harmonic restraint. default unit: kj/mol/nm**2
    cutoff: cutoff distance, unit: nanometer
    return omm system
    """
    internal_force = HarmonicBondForce()
    Kcons_internal = Kcons * unit.kilojoule_per_mole/unit.nanometers**2

    # load pdb to mdtraj
    pdb_md = md.load_pdb(pdb_ref)

    # find all pairs
    pairs = []
    for domain in domain_ranges:
        # get C-alpha atom indices of folder region
        folded_CA_idx = identify_folded_CA_idx(pdb=pdb_md, domain=domain)
        pairs = list(combinations(folded_CA_idx, 2))
    
        pairs_num = 0
        for index in pairs:
            r1=np.squeeze(pdb_md.xyz[:,int(index[0]),:])
            r2=np.squeeze(pdb_md.xyz[:,int(index[1]),:])
            # dist0=np.sqrt((r1[0]-r2[0])**2+(r1[1]-r2[1])**2+(r1[2]-r2[2])**2)
            dist0 = np.linalg.norm(r1-r2)
            if dist0 < 1.2:
                pairs_num += 1
                internal_force.addBond(int(index[0]),int(index[1]), dist0*unit.nanometers, Kcons_internal)
        print(f"Number of internal pairs of domain {str(domain)}: {pairs_num}")
    system.addForce(internal_force)
    return system

In [42]:
domain_ranges = [('A', (1, 132)), ('A', (159, 226)), ('B', (330, 509)), ('B', (510, 654))]

In [49]:
system4 = copy.deepcopy(system)
system4 = domain_3D_restraint(system4, pdb_ref=pdb_file, domain_ranges=domain_ranges, Kcons=400, cutoff=1.2)

# save system xml out
with open(f"system4_domain3d_res_test.xml", "w") as f:
    f.write(XmlSerializer.serialize(system4))

Number of internal pairs of resid ('A', (1, 132)): 1146
Number of internal pairs of resid ('A', (159, 226)): 290
Number of internal pairs of resid ('B', (330, 509)): 965
Number of internal pairs of resid ('B', (510, 654)): 1145
